In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

In [3]:
import os
import torch

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [4]:
import math
import os
from functools import partial

import torch
from datasets import disable_caching
from transformers import HfArgumentParser, set_seed
from transformers.trainer_utils import get_last_checkpoint

import wandb
from rlvsil.dataset.summarisation_dataloader import (make_summarisation_collate_fn, make_summarisation_dataset)
from rlvsil.evaluation.reward_functions import make_reward_function
from rlvsil.evaluation.summarisation_metrics import compute_summarisation_metrics
from rlvsil.lab.args import Args
from rlvsil.lab.sl_trainer import (EvalGenerationTrainer, EvalGenerationTrainingArguments)

import sys

sys.argv = [sys.argv[0]]
sys.argv.append("--output_dir=.")
sys.argv.append("--model_name=huggyllama/llama-7b")
sys.argv.append("--generation_max_new_tokens=48")
sys.argv.append("--training_evaluation_batches=10")
sys.argv.append("--validation_evaluation_batches=20")
sys.argv.append("--fp16=1")
sys.argv.append("--freeze_layers=0.8")
sys.argv.append("--per_device_train_batch_size=1")
sys.argv.append("--gradient_accumulation_steps=16")
sys.argv.append("--per_device_eval_batch_size=1")


parser = HfArgumentParser((Args, EvalGenerationTrainingArguments))
args, training_args = parser.parse_args_into_dataclasses()
args.training = training_args.do_train

/home/djaniak/mambaforge/envs/rlhf-gen-div/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
args.resume

'allow'

In [6]:
args.eval_sl_reward_function

False

In [7]:
from functools import partial

import torch
from datasets import disable_caching
from transformers import HfArgumentParser, set_seed

import wandb
from rlvsil.models.model_creation import (construct_device_map, construct_model_from_class, tie_frozen_layers)
from rlvsil.utils.core import normalise_seed
from rlvsil.utils.hardware import get_device


disable_caching()
device = get_device(args.device_idx)

training_args.seed = normalise_seed(training_args.seed)
set_seed(training_args.seed)

args.device = device

model, tokenizer, model_ref = construct_model_from_class(args)
model.to(device)
if args.eval_rl_model:
    rl_checkpoint = torch.load(args.eval_rl_model_checkpoint_dir)
    model.load_state_dict(rl_checkpoint["learner_state"]["model"])

if model_ref:
    model_ref.to(device)

if args.wandb_watch:
    wandb.watch(model)

trainer_cls = EvalGenerationTrainer

reward_function = None

/home/djaniak/mambaforge/envs/rlhf-gen-div/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/djaniak/mambaforge/envs/rlhf-gen-div/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/djaniak/mambaforge/envs/rlhf-gen-div/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
raw_dataset = make_summarisation_dataset(args, tokenizer)
collate_fn = make_summarisation_collate_fn(args, tokenizer)
metric_fn = partial(compute_summarisation_metrics, tokenizer=tokenizer, reward_function=reward_function)

/home/djaniak/mambaforge/envs/rlhf-gen-div/lib/python3.10/site-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Map:   0%|          | 0/116722 [00:00<?, ? examples/s]

Map:   0%|          | 0/6447 [00:00<?, ? examples/s]

Map:   0%|          | 0/6553 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/116722 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/6447 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/6553 [00:00<?, ? examples/s]

In [9]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary', 'queries', 'outputs', 'input_ids', 'labels'],
        num_rows: 116722
    })
    validation: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary', 'queries', 'outputs', 'input_ids', 'labels'],
        num_rows: 6447
    })
    test: Dataset({
        features: ['id', 'subreddit', 'title', 'post', 'summary', 'queries', 'outputs', 'input_ids', 'labels'],
        num_rows: 6553
    })
})

In [10]:
trainer = trainer_cls(
    model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=raw_dataset["train"],
    eval_dataset=raw_dataset["validation"],
    data_collator=collate_fn,
    compute_metrics=metric_fn,
)

In [11]:
training_args

EvalGenerationTrainingArguments(output_dir='.', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=False, evaluation_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=16, eval_accumulation_steps=None, eval_delay=0, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='./runs/Jul22_18-20-12_argon', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=None, save_safetensors=False, save_on_each

In [12]:
args

Args(wandb_run_name=None, wandb_project=None, wandb_group=None, wandb_entity='denisj7', wandb_watch=False, wandb_tags=None, resume='allow', run_id=None, device_idx=0, device=device(type='cuda', index=0), model_name='huggyllama/llama-7b', base_model_name='huggyllama/llama-7b', freeze_layers=0.8, freeze_lm_head=True, tie_frozen_layers=False, parallelize=False, bettertransformer=False, torchcompile=False, value_head_activation=False, value_normalisation=0.0, value_normalisation_std=1.0, rm_split_percentage=1.0, policy_split_percentage=1.0, max_new_tokens=100, rl_training=False, summarisation_dataset_queries=False, dataset_random_subset=None, dataset_structured_subset=None, eval_dataset=None, max_source_length=1024, max_target_length=1024, dataset='summarisation', rf_model_dir='checkpoints/hf/sentiment/', reward_function='summarisation', eval_sl_reward_function=False, rf_device_idx=0, rf_device='cuda:0', policy_head_device='cuda:0', target_eval_datapoints=500, eval_rl_model=False, eval_rl_

In [13]:
trainer.train()

/home/djaniak/mambaforge/envs/rlhf-gen-div/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: denisj7 (clarin-pl). Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 44.35 GiB total capacity; 43.57 GiB already allocated; 27.19 MiB free; 43.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import transformers

transformers.__version__

'4.36.2'

In [ ]:
!pip install transformers==4.31.0

  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2
